Судебная практика https://sudact.ru/

В составе раздела – обзор решений судов по наиболее популярным правовым вопросам и ситуациям.

По административным делам - https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/

По гражданским делам - https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/

По уголовным делам - https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/

In [1]:
import numpy as np

In [2]:
sections = np.array([
    ('По административным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
    ('По гражданским делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
    ('По уголовным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')
])

In [4]:
sections[:, 1]

array(['https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/'],
      dtype='<U72')

1. Заходим на страницу раздела
2. Получаем список подразделов по применению конкретных норм
3. Получаем список решений 
4. Переходим на следующую страницу
5. Повторяем пункты 3, 4, пока есть страницы

Для конкретного решения у нас есть структурированная информация:
* название (содержит номер и дату)
* ссылка на страницу
* раздел
* подраздел
* суд
* html страницы

In [ ]:
#!pip install scrapy

In [ ]:
#!pip install pyopenssl==22.0.0

In [ ]:
#!pip install crochet

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect("sudactru.db")

cur = con.cursor()

sql = '''
CREATE TABLE IF NOT EXISTS sections(
    id INTEGER PRIMARY KEY,
    name TEXT,
    href TEXT
)
'''

cur.executescript(sql)
con.commit()

In [148]:
#import pandas as pd

In [149]:
sql = '''
INSERT INTO sections(name, href) 
    VALUES (?, ?)
'''
cur.executemany(sql, sections)
con.commit()

In [150]:
res = cur.execute("SELECT * FROM sections;")
res.fetchall()

[(1,
  'По административным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
 (2,
  'По гражданским делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
 (3,
  'По уголовным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')]

In [151]:
class SqliteSectionPipeline:

    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS subsections(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            section_id INT,
            FOREIGN KEY (section_id) REFERENCES sections(id)
        )
        """)


    def process_item(self, item, spider):
        
        self.cur.execute("""
            INSERT INTO subsections (name, href, section_id) VALUES (?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['section_id']
        ))

        self.con.commit()

        return item


In [5]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

In [8]:
class SudactSectionSpider(scrapy.Spider):
    """
    Парсинг списка страниц на применения разных норм в рамках данного раздела.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSectionSpider"
    allowed_domains = ['sudact.ru']
    start_urls = list(sections[:, 1])
    
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()    
        print(h1)
        res = self.cur.execute("SELECT id FROM sections WHERE href = ?", (current_url,))
        section_id = res.fetchone()[0]
            
        for link in response.css('.wos'):
            yield {'name': link.css('::text').get(), 
                   'href': link.css('a').attrib["href"],
                   'section_id': section_id
                  }

@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSectionSpider)
    return d

NameError: name 'sections' is not defined

In [154]:
run_spider()

Судебная практика по административным делам
Судебная практика по гражданским делам
Судебная практика по уголовным делам


In [234]:
res = cur.execute("SELECT href FROM subsections;")
list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))

['https://sudact.ru/practice/amnistiya/',
 'https://sudact.ru/practice/dokazatelstva/',
 'https://sudact.ru/practice/halatnost/',
 'https://sudact.ru/practice/inostrannye-grazhdane/',
 'https://sudact.ru/practice/kleveta/',
 'https://sudact.ru/practice/kommercheskij-podkup/',
 'https://sudact.ru/practice/kontrabanda/',
 'https://sudact.ru/practice/mery-presecheniya/',
 'https://sudact.ru/practice/narushenie-prav-invalidov/',
 'https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/',
 'https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/',
 'https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/',
 'https://sudact.ru/practice/oskorblenie/',
 'https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/',
 'https://sudact.ru/practice/personalnye-dannye/',
 'https://sudact.ru/practice/po-delam-o-huliganstve/',
 'https://sudact.ru/practice/po-delam-ob-iznasilovanii/',
 'https://sudact.ru/practice/po-delam-ob-ubijstve/',
 'https://sudact.ru/practice/po

In [233]:
res = cur.execute("SELECT section_id, COUNT(*) FROM subsections GROUP BY section_id;")
res.fetchall()

[(1, 13), (2, 5), (3, 34)]

In [241]:
class SqliteSubSectionPipeline:
    
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS decisions(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            subsection_id INT,
            justice TEXT,
            short_descr TEXT,
            FOREIGN KEY (subsection_id) REFERENCES subsection(id)
        )
        """)
        # justice - название суда и типа дела, 
        # например "Шатурский городской суд (Московская область) - Гражданское"
        # short_descr - короткое "описание" решения, содержащие название нормы, 
        # в рамках которой рассматривалось данное дело


    def process_item(self, item, spider):
                
        self.cur.execute("""
            INSERT INTO decisions (
                name, 
                href, 
                subsection_id,
                justice,
                short_descr
            ) VALUES (?, ?, ?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['subsection_id'],
            item['justice'],
            item['short_descr']
        ))

        self.con.commit()
        
        return item

In [245]:
class SudactSubSectionSpider(scrapy.Spider):
    """
    Парсинг списка решений по данной правовой норме.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSubSectionSpider"
    allowed_domains = ['sudact.ru']
    
    res = cur.execute("SELECT href FROM subsections;")
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
    #start_urls = ['https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=49']
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSubSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM subsections WHERE href = ?", (
            current_url.replace('https://sudact.ru', '').split('?')[0],
        ))
        subsection_id = res.fetchone()[0]
           
        print(h1, current_url, subsection_id)
        
        for li in response.css('.results li'):
            yield {'name': li.css('a').css('::text').get(), 
                   'href': li.css('a').attrib["href"],
                   'subsection_id': subsection_id,
                   'justice': li.css('.b-justice::text').get(),
                   'short_descr': ''.join(li.css('::text').getall()[3:])
                  }
        
        next_page = response.css('.page-next a')
        if len(next_page) > 0:
            yield scrapy.Request(
                response.urljoin(current_url.split('?')[0] + next_page.attrib["href"]),
                callback=self.parse
            )

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSubSectionSpider)
    return d

In [246]:
run_spider()

Амнистия - судебная практика https://sudact.ru/practice/amnistiya/ 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/ 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/ 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/ 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/ 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/ 29
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/ 40
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/ 52
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/ 3
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/ 49
Незаконное получение кредита - судебная практика https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/ 4
Незаконное предпринимательств

Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=5 7
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=5 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=5 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=6 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=6 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=6 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=6 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=6 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=6 29
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=2 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya

Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=9 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=10 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=10 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/?page=10 5
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=10 51
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=10 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=10 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=11 32
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=10 7
Халатность - судебная практика https://sudact.ru/p

Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=15 19
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=14 7
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=11 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=15 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=15 29
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/?page=13 3
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=14 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=15 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=15 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvleni

Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=19 26
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=18 40
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/?page=19 49
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=20 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=20 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=20 19
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=19 51
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=16 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=20 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=20 29
Нарушение прав инвалидов - судебная п

...

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=18 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=20 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=18 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=16 47
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=16 30
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=19 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=16 22
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=20 25
Преступное сообщество - судебная практ

Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=22 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=27 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=26 25
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=22 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=43 46
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=25 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=27 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=25 45
Умышленное причинение тяжкого 

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=32 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=29 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=34 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=33 25
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=32 20
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=29 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=50 46
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=34 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/so

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=39 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=41 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=39 45
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=36 30
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=37 47
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=40 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=37 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=42 37
Самоуправст

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=47 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=44 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=49 37
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=47 20
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=48 25
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=49 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=47 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=45 47
Уголовная ответственность несовершеннолетних - судебная пр

In [8]:
res = cur.execute("SELECT COUNT(*) FROM decisions;")
res.fetchall()

[(51304,)]

In [270]:
res = cur.execute("SELECT subsection_id, COUNT(*) FROM decisions GROUP BY subsection_id;")
res.fetchall()

[(1, 1000),
 (2, 1270),
 (3, 744),
 (4, 734),
 (5, 1290),
 (6, 1000),
 (7, 1280),
 (8, 1310),
 (9, 1000),
 (10, 1000),
 (11, 1000),
 (12, 1000),
 (13, 1040),
 (14, 22),
 (15, 20),
 (16, 90),
 (17, 22),
 (18, 12),
 (19, 1310),
 (20, 1000),
 (21, 1270),
 (22, 1000),
 (23, 1000),
 (24, 1000),
 (25, 1000),
 (26, 1300),
 (27, 1000),
 (28, 1000),
 (29, 1300),
 (30, 1000),
 (31, 1290),
 (32, 1300),
 (33, 1000),
 (34, 1000),
 (35, 1290),
 (36, 1000),
 (37, 1000),
 (38, 1000),
 (39, 1000),
 (40, 1270),
 (41, 1300),
 (42, 1000),
 (43, 1000),
 (44, 1000),
 (45, 1000),
 (46, 1000),
 (47, 1000),
 (48, 1000),
 (49, 1290),
 (50, 1000),
 (51, 1260),
 (52, 1290)]

In [271]:
res = cur.execute("SELECT * FROM decisions LIMIT 1;")
res.fetchall()

[(1,
  'Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022',
  '/regular/doc/chaeDyxsKiYN/',
  32,
  'Шумихинский районный суд (Курганская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осужденного от отбывания назначенного наказания. Лицо, освобожденное от отбывания наказания по указанным основаниям, считается несудимым (часть 2 статьи 86 УК РФ).\nВ ...\n\n\n')]

In [6]:
class SaverDecisionContentPipeline:
    
    def process_item(self, item, spider):
       
        decision_id = item['decision_id']
        with open(f'decisions__data/{decision_id}.txt', 'w') as f:
            f.write(item['content'])
            
        return item

In [7]:
import os

In [8]:
class SudactDecisionSpider(scrapy.Spider):
    """
    Парсинг содержимого решений в формате html 
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactDecisionSpider"
    allowed_domains = ['sudact.ru']
    
    #res = cur.execute("SELECT href FROM decisions;")
    # выбрали уголовные дела
#     res = cur.execute('''
#         SELECT
#             d.href
#         FROM 
#             decisions AS d
#         INNER JOIN subsections AS ss ON d.subsection_id = ss.id
#         INNER JOIN sections AS s ON s.id = ss.section_id
#         WHERE
#             s.id = 3
#     ''')
    # выбрали все те, которые еще не спаршены
    parsed_decisions_ids = sorted(list(map(lambda x: int(x.replace('.txt', '')),
                           os.listdir('decisions__data'))))
    placeholder = '?' 
    placeholders = ', '.join(placeholder * len(parsed_decisions_ids))

    res = cur.execute('''
        SELECT
            href
        FROM 
            decisions
        WHERE
            id NOT IN(%s)
    ''' % placeholders, tuple(parsed_decisions_ids))
    
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
    print(start_urls[:25])
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SaverDecisionContentPipeline': 1
        },
        "DOWNLOAD_DELAY": 1,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM decisions WHERE href = ?", (
            current_url.replace('https://sudact.ru', ''),
        ))
        decision_id = res.fetchone()[0]
        
        html = response.xpath(r'//*[contains(@class, "h-content-wrap2")]/descendant-or-self::*[not(self::script or self::style)]/text()[normalize-space(.)]')
        text = '\n'.join(
            list(
                map(
                    lambda x: " ".join(
                                x.replace('\n', '').
                                replace('\t', '').
                                replace('\xa0', '').
                                split()
                            ), 
                    html.extract()
                )
            )[3:-10]
        )
        
        print(h1, current_url, decision_id, len(text))
        
        yield {'content': text,
               'decision_id': decision_id}

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactDecisionSpider)
    return d

['https://sudact.ru/regular/doc/3O66Bm0NgJr/', 'https://sudact.ru/regular/doc/y1fYZYAX8aJW/', 'https://sudact.ru/regular/doc/DRTcUvsB2FJn/', 'https://sudact.ru/regular/doc/lEPeLqImqilF/', 'https://sudact.ru/regular/doc/MmROpQeS0vQN/', 'https://sudact.ru/regular/doc/T7mS5NG7Gd1c/', 'https://sudact.ru/regular/doc/cP0FmoWtsYNR/', 'https://sudact.ru/regular/doc/ERib7ttrw9dl/', 'https://sudact.ru/regular/doc/LxO6wGHIavVp/', 'https://sudact.ru/regular/doc/C7wn0rGmhUNj/', 'https://sudact.ru/regular/doc/YTp6AG4M5dZ8/', 'https://sudact.ru/regular/doc/PNtrPX7rkLVQ/', 'https://sudact.ru/regular/doc/iVj5dzgiRlV8/', 'https://sudact.ru/regular/doc/h4ZM37c3ZtQ1/', 'https://sudact.ru/regular/doc/CILOUbuSe8Yi/', 'https://sudact.ru/regular/doc/PCpLuo2wYg6H/', 'https://sudact.ru/regular/doc/IYEjoVKufaZX/', 'https://sudact.ru/regular/doc/4JmKFeqd9Gmj/', 'https://sudact.ru/regular/doc/pF00u2GPh0bO/', 'https://sudact.ru/regular/doc/RGIK9jEiROd3/', 'https://sudact.ru/regular/doc/InU1Sk2QEpdW/', 'https://suda

In [ ]:
run_spider()

Решение № 30-2-517/2021 от 17 августа 2021 г. по делу № 30-2-517/2021 https://sudact.ru/regular/doc/3O66Bm0NgJr/ 3433 7549
Решение № 30-2-516/2021 от 16 августа 2021 г. по делу № 30-2-516/2021 https://sudact.ru/regular/doc/y1fYZYAX8aJW/ 3434 7546
Апелляционное постановление № 22-1461/2021 от 9 августа 2021 г. по делу № 4/17-232/2021 https://sudact.ru/regular/doc/DRTcUvsB2FJn/ 3435 6426
Постановление № 1-103/2021 от 30 июля 2021 г. по делу № 1-103/2021 https://sudact.ru/regular/doc/lEPeLqImqilF/ 3436 6513
Приговор № 1-218/2021 от 29 июля 2021 г. по делу № 1-218/2021 https://sudact.ru/regular/doc/MmROpQeS0vQN/ 3437 14896
Постановление № 5-532/2014 от 21 августа 2014 г. по делу № 5-532/2014 https://sudact.ru/regular/doc/T7mS5NG7Gd1c/ 3662 4438
Апелляционное постановление № 22-272/2020 от 27 мая 2020 г. по делу № 1-147/2019 https://sudact.ru/regular/doc/cP0FmoWtsYNR/ 3861 33472
Решение № 2-353/2020 2-353/2020~М-246/2020 М-246/2020 от 15 июля 2020 г. по делу № 2-353/2020 https://sudact.ru/r

Решение № 12-138/2020 5-99/2020 от 26 мая 2020 г. по делу № 12-138/2020 https://sudact.ru/regular/doc/zrN5LptuGurK/ 9085 18083
Решение № 12-80/2020 от 26 мая 2020 г. по делу № 12-80/2020 https://sudact.ru/regular/doc/9ZJeUwyUZZWW/ 9086 8730
Решение № 2-448/2020 2-448/2020~М-218/2020 М-218/2020 от 25 мая 2020 г. по делу № 2-448/2020 https://sudact.ru/regular/doc/4EdnbQn4j7J0/ 9087 9768
Решение № 12-356/2020 от 25 мая 2020 г. по делу № 12-356/2020 https://sudact.ru/regular/doc/fyAxn6CmGcoM/ 9088 7414
Решение № 12-44/2020 от 25 мая 2020 г. по делу № 12-44/2020 https://sudact.ru/regular/doc/OvS2QVXnkyNu/ 9089 10402
Решение № 12-61/2020 от 25 мая 2020 г. по делу № 12-61/2020 https://sudact.ru/regular/doc/QRaL7i4CAiW5/ 9090 12433
Приговор № 1-422/2020 1-58/2021 от 11 марта 2021 г. по делу № 1-422/2020 https://sudact.ru/regular/doc/BpTGTanty2Ht/ 9131 15272
Приговор № 1-32/2021 1-362/2020 от 11 марта 2021 г. по делу № 1-32/2021 https://sudact.ru/regular/doc/dRDafNAaFfVV/ 9132 30539
Приговор № 

Апелляционное постановление № 22-826/2020 от 9 декабря 2020 г. по делу № 4/16-59/2020 https://sudact.ru/regular/doc/IQ3hpq47oLru/ 9309 15238
Апелляционное постановление № 22-806/2020 22К-806/2020 от 3 декабря 2020 г. по делу № 3/6-51/2020 https://sudact.ru/regular/doc/fhjruyYYi8W4/ 9310 9964
Решение № 2-439/2021 2-439/2021~М-251/2021 М-251/2021 от 16 марта 2021 г. по делу № 2-439/2021 https://sudact.ru/regular/doc/0cjvuZk1Ek14/ 9311 21133
Решение № 2-2030/2020 2-333/2021 2-333/2021(2-2030/2020;)~М-1820/2020 М-1820/2020 от 15 марта 2021 г. по делу № 2-2030/2020 https://sudact.ru/regular/doc/tzWhYjLPpiVm/ 9312 22502
Приговор № 1-40/2021 от 15 марта 2021 г. по делу № 1-40/2021 https://sudact.ru/regular/doc/hKboXOEXLV5m/ 9313 7072
Решение № 12-114/2021 от 15 марта 2021 г. по делу № 12-114/2021 https://sudact.ru/regular/doc/iVMvHnnryf6q/ 9314 14749
Решение № 2-25/2021 2-25/2021(2-394/2020;)~М-400/2020 2-394/2020 М-400/2020 от 11 марта 2021 г. по делу № 2-25/2021 https://sudact.ru/regular/do

Постановление № 1-316/2020 от 26 мая 2020 г. по делу № 1-316/2020 https://sudact.ru/regular/doc/zxZDDzWaCK1p/ 9420 30622
Решение № 12-430/2011 от 21 сентября 2011 г. по делу № 12-430/2011 https://sudact.ru/regular/doc/J62DhECqO6J5/ 9421 10390
Решение № 12-195/2011 от 14 сентября 2011 г. по делу № 12-195/2011 https://sudact.ru/regular/doc/pADvYFZQAgN4/ 9422 6808
Решение № 12-243/2011 от 2 сентября 2011 г. https://sudact.ru/regular/doc/wgETzbCiBK6l/ 9423 23448
Решение № 12-61/2011 от 23 июня 2011 г. по делу № 12-61/2011 https://sudact.ru/regular/doc/aLrQGcJATjoj/ 9424 5567
Решение № 2-2283\15(10) 2-2283\15 М-1651/2011 2-2283/2011~М-1651/2011 2-2283/2011 от 6 июня 2011 г. https://sudact.ru/regular/doc/4FegQCimfTH5/ 9425 11678
Решение № 2-22835 от 6 июня 2011 г. https://sudact.ru/regular/doc/cqPpDwpz3lC/ 9426 11700
Решение № 12-42/2011 от 23 мая 2011 г. по делу № 12-42/2011 https://sudact.ru/regular/doc/oKWjNitVjiUJ/ 9427 7105
Решение № 12-32/2020 от 7 июля 2020 г. по делу № 12-32/2020 htt

Приговор № 1-86/2019 от 24 июля 2019 г. по делу № 1-86/2019 https://sudact.ru/regular/doc/DGP1c1MfRIxP/ 9498 15310
Постановление № 44У-88/2019 44-У-88/2019 4У-641/2019 от 22 июля 2019 г. по делу № 1-303/2015 https://sudact.ru/regular/doc/ztkuoEzKxe60/ 9499 17241
Апелляционное постановление № 22-5332/2019 от 22 июля 2019 г. по делу № 22-5332/2019 https://sudact.ru/regular/doc/nQOxNrGmizx1/ 9500 15032
Приговор № 1-317/2019 от 18 июля 2019 г. по делу № 1-317/2019 https://sudact.ru/regular/doc/EGXlP7LWwSmm/ 9501 12870
Приговор № 1-323/2019 от 16 июля 2019 г. по делу № 1-323/2019 https://sudact.ru/regular/doc/oMCU0mdOD7lT/ 9503 10716
Приговор № 1-112/2019 от 17 июля 2019 г. по делу № 1-354/2017 https://sudact.ru/regular/doc/gIVskG4a3EZw/ 9502 796936
Приговор № 1-5/2019 1-519/2017 1-64/2018 от 15 июля 2019 г. по делу № 1-5/2019 https://sudact.ru/regular/doc/xjj0HTsVex6B/ 9504 244757
Приговор № 1-499/2019 от 15 июля 2019 г. по делу № 1-499/2019 https://sudact.ru/regular/doc/c2OFPWm705Uu/ 9505

...

Постановление № 4У-1353/2016 от 27 апреля 2016 г. по делу № 4У-1353/2016 https://sudact.ru/regular/doc/80R7XnttM8Yx/ 17725 4063
Апелляционное постановление № 22-2051/16 22К-2051/2016 от 27 апреля 2016 г. по делу № 22К-2051/2016 https://sudact.ru/regular/doc/gWvZPh1gcxxj/ 17726 12418
Апелляционное постановление № 22-703/2016 от 27 апреля 2016 г. по делу № 22-703/2016 https://sudact.ru/regular/doc/0aUZg479BDUK/ 17727 6662
Апелляционное постановление № 22-679/2016 от 27 апреля 2016 г. по делу № 22-679/2016 https://sudact.ru/regular/doc/sM9NslhjbZLK/ 17728 6175
Апелляционное постановление № 22-742/2016 от 27 апреля 2016 г. по делу № 22-742/2016 https://sudact.ru/regular/doc/Mnt0TlGKGTLp/ 17729 7874
Апелляционное постановление № 22-624/2020 от 11 августа 2020 г. по делу № 4/16-51/2020 https://sudact.ru/regular/doc/Wr0yZeKQfQbJ/ 17730 12575
Апелляционное постановление № 22-4851/2020 от 11 августа 2020 г. по делу № 1-107/2020 https://sudact.ru/regular/doc/dwdQEout0gJd/ 17731 12243
Апелляционн

Апелляционное постановление № 22-1800/2020 от 29 сентября 2020 г. по делу № 1-34/2020 https://sudact.ru/regular/doc/AJJLoMFVWqoj/ 17788 25848
Апелляционное постановление № 22-2454/2020 от 29 сентября 2020 г. по делу № 4/13-140/2020 https://sudact.ru/regular/doc/6OgOQhAkudxO/ 17789 5513
Постановление № 5-400/2020 от 5 ноября 2020 г. по делу № 5-400/2020 https://sudact.ru/regular/doc/cGWd72mVsnVt/ 17790 4896
Постановление № 5-132/2020 от 5 ноября 2020 г. по делу № 5-95/2020 https://sudact.ru/regular/doc/BcL0KmY1QPoV/ 17791 8906
Постановление № 5-41/2020 от 5 ноября 2020 г. по делу № 5-41/2020 https://sudact.ru/regular/doc/riK51ZT7aXgh/ 17792 9396
Постановление № 5-95/2020 от 5 ноября 2020 г. по делу № 5-95/2020 https://sudact.ru/regular/doc/wYkRzK3TnNxw/ 17793 5020
Постановление № 5-939/2020 от 5 ноября 2020 г. по делу № 5-939/2020 https://sudact.ru/regular/doc/icbsHpS1FZLr/ 17794 13313
Решение № 12-128/2020 от 5 ноября 2020 г. по делу № 5-226/2020 https://sudact.ru/regular/doc/CkmccVcf1

Решение № 12-173/2021 от 6 июля 2021 г. по делу № 12-173/2021 https://sudact.ru/regular/doc/lHPKvsuaim1T/ 18013 6082
Решение № 12-144/2021 от 5 июля 2021 г. по делу № 12-144/2021 https://sudact.ru/regular/doc/v8mopLUpiboJ/ 18014 15367
Приговор № 1-35/2021 от 5 июля 2021 г. по делу № 1-35/2021 https://sudact.ru/regular/doc/oET6h9XZcwru/ 18016 46522
Решение № 12-135/2021 от 5 июля 2021 г. по делу № 12-135/2021 https://sudact.ru/regular/doc/BB9y2F6oxCNt/ 18017 8052
Решение № 2А-467/2021 2А-467/2021~М-333/2021 М-333/2021 от 2 июля 2021 г. по делу № 2А-467/2021 https://sudact.ru/regular/doc/BZHCeuqcEWmv/ 18018 13176
Решение № 2|2|-12/2021 2|2|-12/2021(2|2|-239/2020;)~М|2|-264/2020 2|2|-239/2020 М|2|-264/2020 от 2 июля 2021 г. по делу № 2|2|-12/2021 https://sudact.ru/regular/doc/mDNzTOzSLBbn/ 18019 33740
Приговор № 1-175/2021 от 7 июля 2021 г. по делу № 1-175/2021 https://sudact.ru/regular/doc/Yqv8MsEQ1cld/ 18020 34919
Приговор № 1-158/2021 от 7 июля 2021 г. по делу № 1-158/2021 https://suda

Решение № 2-8265/2020 2-8265/2020~М-8735/2020 М-8735/2020 от 26 ноября 2020 г. по делу № 2-8265/2020 https://sudact.ru/regular/doc/ZyMlIU8q2jLb/ 18084 13106
Решение № 2-1488/2020 2-1488/2020~М-1313/2020 М-1313/2020 от 26 ноября 2020 г. по делу № 2-1488/2020 https://sudact.ru/regular/doc/AMSxMkzDlWYQ/ 18085 15501
Решение № 2-5565/2020 2-5565/2020~М-5688/2020 М-5688/2020 от 26 ноября 2020 г. по делу № 2-5565/2020 https://sudact.ru/regular/doc/gN9R6PH7i2bT/ 18086 13132
Решение № 2-237/2020 2-237/2020~М-225/2020 М-225/2020 от 26 ноября 2020 г. по делу № 2-237/2020 https://sudact.ru/regular/doc/Evmj5g3q2gou/ 18087 26398
Решение № 21-477/2020 от 25 ноября 2020 г. по делу № 21-477/2020 https://sudact.ru/regular/doc/PGesSBRS74or/ 18089 19238
Апелляционное постановление № 22К-876/2020 от 20 августа 2020 г. по делу № 3/10-2/2020 https://sudact.ru/regular/doc/PojrcyDmDpW0/ 18090 18170
Апелляционное постановление № 22-1279/2020 от 20 августа 2020 г. по делу № 1-199/2020 https://sudact.ru/regular/d

Апелляционное постановление № 22К-404/2020 от 25 мая 2020 г. по делу № 3/14-1/2020 https://sudact.ru/regular/doc/Uaw8LIHsKhxn/ 18144 15782
Решение № 2А-246/2020 2А-246/2020~М-181/2020 М-181/2020 от 22 мая 2020 г. по делу № 2А-246/2020 https://sudact.ru/regular/doc/UoHvcGsoV6WQ/ 18145 17852
Апелляционное постановление № 22-2626/2020 от 21 мая 2020 г. по делу № 4/1-67/2020 https://sudact.ru/regular/doc/9ic683DpimQB/ 18146 7303
Решение № 2А-2391/2020 2А-2391/2020~М-1563/2020 М-1563/2020 от 21 мая 2020 г. по делу № 2А-2391/2020 https://sudact.ru/regular/doc/DvHonZQ6iVVv/ 18147 10215
Апелляционное постановление № 22-492/2020 от 20 мая 2020 г. по делу № 1-5/2020 https://sudact.ru/regular/doc/FII3kgLjH5dZ/ 18148 6761
Решение № 2-59/2020 2-59/2020(2-599/2019;)~М-572/2019 2-599/2019 М-572/2019 от 20 мая 2020 г. по делу № 2-59/2020 https://sudact.ru/regular/doc/qjdX7IndDnbN/ 18149 21214
Решение № 2А-205/2022 2А-205/2022~М-121/2022 М-121/2022 от 24 февраля 2022 г. по делу № 2А-205/2022 https://su

In [95]:
1+1

2

Решение № 2А-1772/2020 2А-1772/2020~М-1668/2020 М-1668/2020 от 29 июля 2020 г. по делу № 2А-1772/2020 https://sudact.ru/regular/doc/xiCJSqLkFogs/ 4674 18087
Решение № 2-527/2013 от 5 марта 2013 г. https://sudact.ru/regular/doc/YuASsqyfJYUr/ 4690 18136
{'content': 'Суть спора: О признании движимой вещи безхозяйной и признании права муниципальной собственности на безхозяйную недвижимую вещь\n№ 2-527/2013\nРЕШЕНИЕ\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\n05 марта 2013 года г. Иркутск\nКуйбышевский районный суд города Иркутска в составе:\nпредседательствующего судьи Глуховой Т.Н.,\nпри секретаре Кармадановой Е.А.,\nс участием представителя заявителя Клячиной О.И., представителя заинтересованного лица Хохрякова В.Б., рассмотрев в открытом судебном заседании гражданское дело по заявлению прокурора Ленинского района г. Иркутска в интересах Российской Федерации о признании имущества бесхозяйным,\nУСТАНОВИЛ:\nПрокурор Ленинского района г. Иркутска обратился в суд с заявлением в интересах Российской Федера

Приговор № 1-532/2020 от 29 июля 2020 г. по делу № 1-532/2020 https://sudact.ru/regular/doc/zRKMGqLCKfYl/ 4675 9982
Приговор № 1-64/2019 от 12 августа 2019 г. по делу № 1-64/2019 https://sudact.ru/regular/doc/k3qeoZoxBDJN/ 4893 23993
{'content': '\ufeffДело № 1-64/2019\n34RS0030-01-2019-000441-70\nПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nг. Новоаннинский «12» августа 2019 года\nНовоаннинский районный суд Волгоградской области в составе председательствующего судьи Захарова Р.П.,\nпри секретаре судебного заседания Пряхиной И.А.,\nс участием государственного обвинителя – помощника прокурора Новоаннинского района Волгоградской области Серединцева С.Н.,\nподсудимого Маренкова Б.Б.,\nзащитника – адвоката Бутурлакина С.А., представившего удостоверение № 1829 и ордер на защиту в порядке ст.\n51 УПК РФ\n№ 045494 от 26.07.2019 года,\nрассмотрев в открытом судебном заседании в общем порядке судебного разбирательства уголовное дело в отношении\nМаренкова Б.Б., ранее судимого: - 27.02.2019 года Новоа

Приговор № 1-315/2020 от 29 июля 2020 г. по делу № 1-315/2020 https://sudact.ru/regular/doc/jE9XRqyEwQVc/ 4676 21990
Приговор № 1-105/2019 от 11 августа 2019 г. по делу № 1-105/2019 https://sudact.ru/regular/doc/NKEUTlDOjVgV/ 4894 13046
{'content': '\ufeffФИО10\nПРИГОВОР\nИФИО1\nФИО11 августа 2019 г.\nКраснослободский районный суд в составе:\nпредседательствующего судьи Гудковой Е.С.,\nпри секретаре ФИО3,\nс участием государственного обвинителя – старшего помощника прокурора ФИО4, подсудимого ФИО2,\nзащитника – адвоката ФИО5,\nпотерпевшей Потерпевший №1,\nрассмотрев в открытом судебном заседании материалы уголовного дела в отношении\nФИО2, родившегося ФИО12 марта 1982 года в Республики Беларусь, гражданина Российской Федерации, со средним общим образованием, не состоящего в браке, невоеннообязанного, работающего в ООО «Бэта-М» специалистом, имеющего на иждивении малолетнего ребенка ФИО13 года рождения, судимого приговором Краснослободского районного суда от ФИО14 июня 2019 года по п. «

Приговор № 1-110/2020 от 29 июля 2020 г. по делу № 1-110/2020 https://sudact.ru/regular/doc/u1ivPLKOCXgP/ 4677 12525
Приговор № 1-161/2019 от 9 августа 2019 г. по делу № 1-161/2019 https://sudact.ru/regular/doc/PZ2q2ipSW4Nu/ 4895 8974
{'content': '\ufeffДело № 1-161/2019\nПРИГОВОР\nИменем Российской Федерации\nг. Исилькуль 09 августа 2019 года\nИсилькульский городской суд Омской области в составе: председательствующего судьи Обуховой В.М., с участием государственного обвинителя Батищевой Е.Ю., подсудимого Илларионова В.Р., защитника Левочкина В.В. при секретаре Быстрицкой Н.В., рассмотрев в открытом судебном заседании материалы уголовного дела в отношении\nИлларионов ВР,\nобвиняемого в совершении преступления, предусмотренного п. «б» ч. 2 ст.\n158 УК РФ\n,\nУСТАНОВИЛ\nВ период с ДД.ММ.ГГГГ точное время и дата не установлены в Илларионов ВР умышленно, из корыстных побуждений с целью хищения чужого имущества, убедившись что за его преступными действиями никто не наблюдает, действуя с еди

...

Приговор № 1-685/2019 от 7 августа 2019 г. по делу № 1-685/2019 https://sudact.ru/regular/doc/sOwfTlPoxzxH/ 4900 47589
{'content': '\ufeffДело № 1-685/2019\nПРИГОВОР\nИменем Российской Федерации\nСанкт-Петербург 07 августа 2019 года\nФрунзенский районный суд Санкт-Петербурга в составе:\nПредседательствующего – судьи Кочетова И.Н.,\nгосударственного обвинителя – помощника прокурора Фрунзенского района Санкт-Петербурга Суева Е.О., подсудимого Власова Н.И., защитника – адвоката Черноусова А.В., представителя МИФНС № 27 Санкт-Петербурга Чернышовой Т.Д., при секретаре Вишняковой М.С.,\nрассмотрев в открытом судебном заседании в особом порядке материалы уголовного дела в отношении:\nВласова Николая Ивановича, ДД.ММ.ГГГГ, уроженца дер. Узкая Губа Дедовичского района Псковской области, гражданина РФ, зарегистрированного и проживающего по адресу: с высшим образованием, работающего в ЗАО «Управление – 15 Метрострой», генеральным директором, женатого, несовершеннолетних детей не имеющего, не суди

Решение № 2-666/2013 2-666/2013~М-414/2013 М-414/2013 от 13 мая 2013 г. https://sudact.ru/regular/doc/lgRisE1z4wLv/ 4683 56028
Решение № 7-128/2020 от 27 ноября 2020 г. по делу № 7-128/2020 https://sudact.ru/regular/doc/iUFGUhggzXoQ/ 4901 7099
{'content': '\ufeff7-128/2020 судья Макарова Н.А.\nРЕШЕНИЕ\nг. Владимир 27 ноября 2020 г.\nСудья Владимирского областного суда Шайкин Ю.А., рассмотрев в открытом судебном заседании жалобу Чупонова Б.С. на постановление судьи Ковровского городского суда Владимирской области от 17 ноября 2020 г., вынесенное в отношении Чупонова Бахмана Саъдиевича, по делу об административном правонарушении, предусмотренном ч.1.1 ст.\n18.8 КоАП\nРФ,\nУСТАНОВИЛ:\nпостановлением судьи Ковровского суда Владимирской области от 17 ноября 2020 г. Чупонов Б.С. признан виновным в совершении административного правонарушения, предусмотренного ч.1.1 ст.\n18.8 КоАП\nРФ. Ему назначено наказание в виде административного штрафа в размере 3000 рублей с административным выдворением 

Постановление № 5-67/2013 от 18 апреля 2013 г. https://sudact.ru/regular/doc/J0rU968reZJq/ 4685 4407
Постановление № 5-667/2020 от 27 ноября 2020 г. по делу № 5-667/2020 https://sudact.ru/regular/doc/CAmytD5UitH/ 4903 7030
{'content': '\ufeff5-667/2020\nП О С Т А Н О В Л Е Н И Е\n27 ноября 2020 года\nСудья Буйского районного суда Беляев В.М.\nрассмотрев в судебном заседании административное дело, возбужденное по ч.1 ст.\n20.6.1 КоАП\nРФ в отношении Смирнова Романа Евгеньевича, родившегося ДД.ММ.ГГГГ в в , зарегистрированного и проживающего по адресу: , д. Юрецкие, , ранее не привлекавшегося к административной ответственности за совершение однородных административных правонарушений,\nУ С Т А Н О В И Л:\nДД.ММ.ГГГГ в 15 часов 40 минут Смирнов Р.Е. находился в помещении магазина ТЦ «Центральный» по адресу Костромская область г. Буй ул. Октябрьской революции д. 42 с пребыванием людей, без использования средств индивидуальной защиты органов дыхания (защитная маска, респиратор), чем допустил

Постановление № 5-66/2013 от 18 апреля 2013 г. https://sudact.ru/regular/doc/hKqldEDVa61x/ 4687 4504
Постановление № 5-325/2020 от 27 ноября 2020 г. по делу № 5-325/2020 https://sudact.ru/regular/doc/4mO4VS5klXVc/ 4905 18345
{'content': '\ufeff№ 5-325/2020\nУИД-37RS0017-01-2020-000894-20\nПОСТАНОВЛЕНИЕ\nпо делу об административном правонарушении\n27 ноября 2020 года город Родники\nСудья Родниковского районного суда Ивановской области Кузнецова Е.В.,\nс участием лица, в отношении которого ведется производство по делу об административном правонарушении, А.З.З.К.,\nлица, составившего протокол об административном правонарушении начальника отделения по вопросам миграции МО МВД России «Родниковский» К.Н.С.,\nрассмотрев в открытом судебном заседании дело об административном правонарушении, предусмотренном ч. 1.1 ст.\n18.8 КоАП\nРФ в отношении А.З.З.К., ДД.ММ.ГГГГ года рождения, уроженки Республики Азербайджан, русским языком владеющей, регистрации на территории РФ не имеющей, фактически пребы

Определение № 5-16/2013 от 10 апреля 2013 г. https://sudact.ru/regular/doc/k9FINVNr2gYq/ 4688 5358
Постановление № 5-2366/2020 от 27 ноября 2020 г. по делу № 5-2366/2020 https://sudact.ru/regular/doc/Sn3IymwVnhlg/ 4906 4459
{'content': '\ufeff\nПОСТАНОВЛЕНИЕ\n27 ноября 2020\nСудья Мытищинского городского суда Дементьева И.В., рассмотрев дело об административном правонарушении в отношении гражданина Республики , ДД.ММ.ГГГГ г.р., паспорт: № выдан ДД.ММ.ГГГГ МВФИО2, проживающего по адресу: по месту пребывания не состоит, пребывающего по адресу: М.О., , территория места пребывания (проживания) иностранных граждан, привлекаемого к административной ответственности, предусмотренной ч. 3.1 ст.\n18.8\nКодекса Российской Федерации об административных правонарушениях,\nУСТАНОВИЛ:\nФИО5 нарушил режим пребывания иностранного гражданина в Российской Федерации в , что выразилось в уклонении от выезда из Российской Федерации по истечении определенного срока пребывания, при следующих обстоятельствах:\n

In [75]:
# удаляем дубликаты decisions, т.к. unique constrant почему-то не отработал...

In [77]:
res = cur.execute('''
    SELECT
        id, 
        href,
        COUNT(*) c
    FROM 
        decisions
    GROUP BY href 
    HAVING c > 1
''')
duplicates = res.fetchall()

In [78]:
duplicates

[(23145, '/practice/alimenty-v-tverdoj-denezhnoj-summe/', 2),
 (23127, '/practice/dobrosovestnyj-priobretatel/', 2),
 (23119, '/practice/dogovor-renty/', 2),
 (23108, '/practice/dolg-po-raspiske-po-dogovoru-zajma/', 2),
 (23304, '/practice/grazhdansko-pravovoj-dogovor/', 2),
 (23125, '/practice/iskovaya-davnost-po-srokam-davnosti/', 2),
 (23114, '/practice/istochnik-povyshennoj-opasnosti/', 2),
 (23315, '/practice/kapitalnyj-remont/', 2),
 (23147, '/practice/lishenie-roditelskih-prav-otca/', 2),
 (23307, '/practice/materialnaya-otvetstvennost/', 2),
 (23103, '/practice/mnimye-sdelki/', 2),
 (23090, '/practice/moralnyj-vred-i-ego-kompensaciya-vozmeshenie-moral/', 2),
 (23104, '/practice/nedostojnyj-naslednik/', 2),
 (23118, '/practice/nedvizhimoe-imushestvo-samovolnye-postrojki/', 2),
 (23097, '/practice/neosnovatelnoe-obogashenie-vzyskanie-neosnovatelno/', 2),
 (23321, '/practice/nezhilye-pomesheniya/', 2),
 (23098, '/practice/obshaya-sobstvennost-opredelenie-dolej-v-obshej-so/', 2),
 

In [80]:
res = cur.executescript('''
    DELETE FROM decisions
    WHERE ID NOT IN
    (
        SELECT MIN(id)
        FROM
            decisions
        GROUP BY href
    )
''')
con.commit()

In [81]:
res = cur.execute("SELECT COUNT(*) FROM decisions;")
res.fetchall()

[(20654,)]

In [35]:
1+1

2

Апелляционное постановление № 22-1744/2022 от 16 августа 2022 г. по делу № 4/1-30/2022 https://sudact.ru/regular/doc/ddu1rIxSHTYr/ 51 4398
{'content': 'Судья 1-й инстанции Лапшин С.В. Дело № 22-1744/22\nАПЕЛЛЯЦИОННОЕ ПОСТАНОВЛЕНИЕ\nг. Ярославль 16 августа 2022 года\nЯрославский областной суд в составе:\nпредседательствующего судьи Предко И.П.,\nпри помощнике судьи Крюковой Ю.С.,\nрассмотрел в открытом судебном заседании апелляционную жалобу адвоката Харламова О.А. на постановление Заволжского районного суда г. Ярославля от 22 июня 2022 года, которым\nотказано в принятии к рассмотрению ходатайства адвоката Харламова О.А. в интересах осуждённого Игнатьева О.А. об условно-досрочном освобождении от отбывания дополнительного наказания в виде штрафа, постановлено ходатайство возвратить адвокату.\nЗаслушав адвоката Харламова О.А. в поддержание доводов апелляционной жалобы, мнение прокурора Хлупиной А.В., полагавшей постановление суда отменить, суд,\nу с т а н о в и л :\nПриговором Фрунзенског

Решение № 2А-309/2022 2А-309/2022~М-139/2022 М-139/2022 от 9 февраля 2022 г. по делу № 2А-309/2022 https://sudact.ru/regular/doc/crZeMVgimeYw/ 54 18118
{'content': 'Дело № 2а-309/2022\nРЕШЕНИЕ\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\n09 февраля 2022 года г.Шилка\nСудья Шилкинского районного суда Сатина И.П., рассмотрев в порядке упрощенного (письменного) производства гражданское дело по административному исковому заявлению заместителя Шилкинского межрайонного прокурора к Управлению Федеральной службы по надзору в сфере связи, информационных технологий и массовых коммуникаций по Забайкальскому краю о признании информации запрещенной,\nУ С Т А Н О В И Л:\nИстец обратился в суд с вышеуказанным заявлением, ссылаясь на то, что по результатам проведенной проверки соблюдения законодательства о защите детей от информации, причиняющей вред их здоровью, выявлен факт размещения в информационно-телекоммуникационной сети «Интернет» для свободного доступа неограниченного круга лиц информации о продаже военного

Апелляционное постановление № 22-1787/2021 от 21 сентября 2021 г. по делу № 4/9-2/2021 https://sudact.ru/regular/doc/LG8TfFsADI5R/ 55 4199
{'content': '\ufeffСудья 1-й инстанции Ятманова А.Ю. Дело № 22-1787/21\nАПЕЛЛЯЦИОННОЕ ПОСТАНОВЛЕНИЕ\nг. Ярославль 21 сентября 2021 года\nЯрославский областной суд в составе:\nпредседательствующего судьи Предко И.П.,\nпри помощнике судьи Коноваловой Е.В.,\nрассмотрел в открытом судебном заседании апелляционную жалобу осужденного Павлова А.Г. на постановление Ленинского районного суда г. Ярославля от 3 августа 2021 года, которым\nоставлено без удовлетворения ходатайство осуждённого Павлова Андрея Георгиевича о рассрочке уплаты штрафа, назначенного по приговору Ленинского районного суда г. Ярославля от 18 января 2021 года.\nЗаслушав осуждённого Павлова А.Г. и адвоката Пеунова В.В. в поддержание доводов апелляционной жалобы, мнение прокурора Докучаевой М.Б., полагавшей постановление суда оставить без изменения, суд,\nу с т а н о в и л :\nПриговором Лени

Приговор № 1-63/2021 от 12 июля 2021 г. по делу № 1-63/2021 https://sudact.ru/regular/doc/SAbRqxcFEAgq/ 56 74897
{'content': '\ufeffДело № 1-63/2021 УИД : 66 RS 0060-01-2021-000189-20\nП Р И Г О В О Р\nИменем Российской Федерации\n12 июля 2021 года п.г.т. Шаля Свердловской области\nШалинский районный суд Свердловской области в составе председательствующего судьи Мироновой С.Н.,\nпри секретаре Рогожниковой В.А.,\nс участием государственного обвинителя Коршакевича Д.А.,\nподсудимой К.Н.А. и ее защитника З.,\nрассмотрев в открытом судебном заседании в общем порядке материалы уголовного дела в отношении:\nК.Н.А., не судимой,\nмера пресечения в виде подписки о невыезде и надлежащем поведении,\nобвиняемой в совершении преступления, предусмотренного ч.3 ст.\n30\n, п. «б» ч.3 ст.\n204\nУголовного кодекса Российской Федерации,\nу с т а н о в и л :\nК.Н.А. совершила покушение на коммерческий подкуп, то есть умышленные действия лица, непосредственно направленные на незаконную передачу лицу, выпол

Решение № 1-1/2021 1-59/2020 от 10 марта 2021 г. по делу № 1-1/2021 https://sudact.ru/regular/doc/Xk80WK5SEux1/ 57 36957
{'content': '\ufeffУИД 62RS0031-01-2020-000198-52\nДело № 1-1/2021\n10 марта 2021 года р.п. Шилово Рязанской области\nШиловский районный суд Рязанской области в составе председательствующего судьи Кораблевой М.А., с участием государственного обвинителя – заместителя прокурора Шиловского района Рязанской области Вырнава Б.М., подсудимой Петрушкиной У.В., защитника - адвоката коллегии адвокатов «Защитник» Адвокатской палаты Рязанской области Бодько А.А. при секретаре Савостиковой М.В., рассмотрев в открытом судебном заседании в зале суда материалы уголовного дела в отношении:\nПетрушкиной Ульяны Владимировны,\nобвиняемой в совершении преступления, предусмотренного ч.2 ст.\n291.1 УК РФ\n,\nУ С Т А Н О В И Л:\nПетрушкина У.В. обвиняется в совершении посредничества во взяточничестве, то есть непосредственной передаче взятки по поручению взяткополучателя в значительном раз

Апелляционное постановление № 22-3121/2020 от 9 декабря 2020 г. по делу № 4/17(1)-22/2020 https://sudact.ru/regular/doc/EhONQPlj1MYP/ 58 17301
{'content': '\ufeffСудья Ильина Е.В. № 22-3121/2020\nАПЕЛЛЯЦИОННОЕ ПОСТАНОВЛЕНИЕ\nг. Оренбург 9 декабря 2020 года\nОренбургский областной суд в составе председательствующего судьи – Баранова С.Б.,\nс участием прокурора отдела прокуратуры Оренбургской области – Горяиновой М.С.,\nзаявителя ФИО1,\nпри секретаре судебного заседания Гапкаловой Н.В.,\nрассмотрел в открытом судебном заседании материал по апелляционной жалобе представителя Министерства финансов Российской Федерации в лице Управления Федерального казначейства по Оренбургской области – ФИО2 на постановление Абдулинского районного суда Оренбургской области от 7 октября 2020 года об удовлетворении ходатайства ФИО1 о возмещении материального ущерба, связанного с незаконным уголовным преследованием.\nЗаслушав доклад судьи Баранова С.Б., пояснения заявителя ЧетверкинаА.Ю., возражавшего против 

In [69]:
res = cur.execute('''
    SELECT
        id, 
        href,
        COUNT(*) c
    FROM 
        decisions
    GROUP BY href 
    HAVING c > 1
''')

In [70]:
res.fetchall()

[(23145, '/practice/alimenty-v-tverdoj-denezhnoj-summe/', 2),
 (23127, '/practice/dobrosovestnyj-priobretatel/', 2),
 (23119, '/practice/dogovor-renty/', 2),
 (23108, '/practice/dolg-po-raspiske-po-dogovoru-zajma/', 2),
 (23304, '/practice/grazhdansko-pravovoj-dogovor/', 2),
 (23125, '/practice/iskovaya-davnost-po-srokam-davnosti/', 2),
 (23114, '/practice/istochnik-povyshennoj-opasnosti/', 2),
 (23315, '/practice/kapitalnyj-remont/', 2),
 (23147, '/practice/lishenie-roditelskih-prav-otca/', 2),
 (23307, '/practice/materialnaya-otvetstvennost/', 2),
 (23103, '/practice/mnimye-sdelki/', 2),
 (23090, '/practice/moralnyj-vred-i-ego-kompensaciya-vozmeshenie-moral/', 2),
 (23104, '/practice/nedostojnyj-naslednik/', 2),
 (23118, '/practice/nedvizhimoe-imushestvo-samovolnye-postrojki/', 2),
 (23097, '/practice/neosnovatelnoe-obogashenie-vzyskanie-neosnovatelno/', 2),
 (23321, '/practice/nezhilye-pomesheniya/', 2),
 (23098, '/practice/obshaya-sobstvennost-opredelenie-dolej-v-obshej-so/', 2),
 

In [74]:
res = cur.execute('select * from decisions where href = ?', 
                  ('/practice/alimenty-v-tverdoj-denezhnoj-summe/',)
                 )
res.fetchall()

[(23145,
  'Алименты в твердой денежной сумме',
  '/practice/alimenty-v-tverdoj-denezhnoj-summe/',
  15,
  None,
  ''),
 (46157,
  'Алименты в твердой денежной сумме',
  '/practice/alimenty-v-tverdoj-denezhnoj-summe/',
  15,
  None,
  '')]

In [73]:
cur.execute("""
            INSERT INTO decisions (
                name, 
                href, 
                subsection_id,
                justice,
                short_descr
            ) VALUES (?, ?, ?, ?, ?)
        """,
        (
            'test',
            '/regular/doc/XLvrx87Jir5u/',
            51,
            '',
            ''
        ))

In [39]:
parsed_decisions_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 1

In [71]:
res = cur.execute('''
    SELECT
        *
    FROM 
        decisions
''')

In [ ]:
(13,
  'Приговор № 1-39/2022 от 24 февраля 2022 г. по делу № 1-39/2022',
  '/regular/doc/XLvrx87Jir5u/',
  51,
  'Шебекинский районный суд (Белгородская область) - Уголовное\n\n\n',
  '\n\n...Шебекинскому городскому округу ФИО9 и оперуполномоченного ОНК ОМВД России по Шебекинскому городскому округу ФИО14 №, так как данные документы не соответствуют требованиям, предъявляемым ч. 1 ст.   74   УПК РФ к доказательствам по уголовному делу.\nРезультаты проведенного по уголовному делу личного досмотра ФИО1 ФИО54 суд признает допустимым доказательством по делу, так как личный досмотр проведен ...\n\n\n')

In [72]:
hrefs = res.fetchall() 
hrefs

[(1,
  'Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022',
  '/regular/doc/chaeDyxsKiYN/',
  32,
  'Шумихинский районный суд (Курганская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осужденного от отбывания назначенного наказания. Лицо, освобожденное от отбывания наказания по указанным основаниям, считается несудимым (часть 2 статьи 86 УК РФ).\nВ ...\n\n\n'),
 (2,
  'Приговор № 1-2/2021 1-55/2020 от 17 июня 2021 г. по делу № 1-2/2021',
  '/regular/doc/NbWQYxKHSxrH/',
  32,
  'Чухломский районный суд (Костромская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осу

In [43]:
13 in parsed_decisions_ids

True

In [46]:
[href[1] for href in [(1, '123'), (2, '456')] if href[0] not in [1]]

['456']

In [50]:
parsed_decisions_ids = sorted(list(map(lambda x: int(x.replace('.txt', '')),
                       os.listdir('decisions__data'))))
res = cur.execute('''
    SELECT
        id, href
    FROM 
        decisions
''')

In [51]:
hrefs = res.fetchall()
hrefs

[(1, '/regular/doc/chaeDyxsKiYN/'),
 (2, '/regular/doc/NbWQYxKHSxrH/'),
 (3, '/regular/doc/b7KFYoe6ITYT/'),
 (4, '/regular/doc/piF7SNfrN6Zm/'),
 (5, '/regular/doc/4uS742mncPV5/'),
 (6, '/regular/doc/h0zs5sMOlsJv/'),
 (7, '/regular/doc/Z3pBtpg0WrlJ/'),
 (8, '/regular/doc/kU0i9wcvEjQt/'),
 (9, '/regular/doc/hZjRLKrnmDVQ/'),
 (10, '/regular/doc/U1nQj4k0YUWZ/'),
 (11, '/regular/doc/9bJfDKBiWQ6P/'),
 (12, '/regular/doc/FVq1nPAGztWl/'),
 (13, '/regular/doc/XLvrx87Jir5u/'),
 (14, '/regular/doc/JrK4LY3jB7Ln/'),
 (15, '/regular/doc/OtxvxKab3lQ1/'),
 (16, '/regular/doc/R26YFK6nNoWX/'),
 (17, '/regular/doc/0DIKIYfjiKgr/'),
 (18, '/regular/doc/rytByFnoNErl/'),
 (19, '/regular/doc/hL2cLZs8Y9Zx/'),
 (20, '/regular/doc/8oZSf4Uul8bJ/'),
 (21, '/regular/doc/nZUa7ipDElNK/'),
 (22, '/regular/doc/hwjxMw18yM19/'),
 (23, '/regular/doc/7zkPgNADqrHU/'),
 (24, '/regular/doc/NUi4FbnaZir5/'),
 (25, '/regular/doc/qfsdwTAOnEQm/'),
 (26, '/regular/doc/7JESAGrGNDma/'),
 (27, '/regular/doc/n798aRyIGzLo/'),
 (28, '/re

In [56]:
hrefs[12]

(13, '/regular/doc/XLvrx87Jir5u/')

In [58]:
hrefs[62]

(63, '/regular/doc/XLvrx87Jir5u/')

In [59]:
hrefs[12][1] == hrefs[62][1]

True

In [ ]:
'/regular/doc/XLvrx87Jir5u/'
'/regular/doc/XLvrx87Jir5u/'

In [54]:
[(href[0], href[1])  for href in hrefs
 if href[0] not in parsed_decisions_ids]

[(63, '/regular/doc/XLvrx87Jir5u/'),
 (72, '/regular/doc/OtxvxKab3lQ1/'),
 (163, '/regular/doc/8lBcEcgz1Mr/'),
 (201, '/regular/doc/40Epyh4sg75r/'),
 (221, '/regular/doc/ehYuSYe6TBoR/'),
 (334, '/regular/doc/XVsBJDilP1LR/'),
 (403, '/regular/doc/hL2cLZs8Y9Zx/'),
 (477, '/regular/doc/OHqfWtvh63Ul/'),
 (501, '/regular/doc/2Ar3OJAaEm6P/'),
 (502, '/regular/doc/eQ7FOS6lVCNn/'),
 (532, '/regular/doc/eYp6DuYAJgZx/'),
 (641, '/regular/doc/7Ys8zVhI0hl4/'),
 (805, '/regular/doc/XcZZcXojMsl8/'),
 (826, '/regular/doc/SmBAx6y4ZKNp/'),
 (841, '/regular/doc/0jXKIhksC8Qx/'),
 (842, '/regular/doc/d8GdLsdTZTgX/'),
 (985, '/regular/doc/hDoZ73w99g1g/'),
 (987, '/regular/doc/LqdfXu6vAB54/'),
 (988, '/regular/doc/YTeSZhbaiL63/'),
 (1041, '/regular/doc/BhcC1qIPg7u/'),
 (1042, '/regular/doc/AQpQjhr3iAW8/'),
 (1043, '/regular/doc/FE5QAdfmqYJl/'),
 (1121, '/regular/doc/chaeDyxsKiYN/'),
 (1122, '/regular/doc/NbWQYxKHSxrH/'),
 (1123, '/regular/doc/b7KFYoe6ITYT/'),
 (1124, '/regular/doc/piF7SNfrN6Zm/'),
 (1125, '

In [ ]:

start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
start_urls[:25]

In [34]:
# parsed_decisions_ids = sorted(list(map(lambda x: int(x.replace('.txt', '')),
#                            os.listdir('decisions__data'))))
# placeholder = '?' 
# placeholders = ', '.join(placeholder * len(parsed_decisions_ids))

# res = cur.execute('''
#     SELECT
#         href
#     FROM 
#         decisions
#     WHERE
#         id NOT IN(%s)
# ''' % placeholders, tuple(parsed_decisions_ids))

# start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
# print(start_urls[:25])

['https://sudact.ru/regular/doc/XLvrx87Jir5u/', 'https://sudact.ru/regular/doc/OtxvxKab3lQ1/', 'https://sudact.ru/regular/doc/8lBcEcgz1Mr/', 'https://sudact.ru/regular/doc/40Epyh4sg75r/', 'https://sudact.ru/regular/doc/ehYuSYe6TBoR/', 'https://sudact.ru/regular/doc/XVsBJDilP1LR/', 'https://sudact.ru/regular/doc/hL2cLZs8Y9Zx/', 'https://sudact.ru/regular/doc/OHqfWtvh63Ul/', 'https://sudact.ru/regular/doc/2Ar3OJAaEm6P/', 'https://sudact.ru/regular/doc/eQ7FOS6lVCNn/', 'https://sudact.ru/regular/doc/eYp6DuYAJgZx/', 'https://sudact.ru/regular/doc/7Ys8zVhI0hl4/', 'https://sudact.ru/regular/doc/XcZZcXojMsl8/', 'https://sudact.ru/regular/doc/SmBAx6y4ZKNp/', 'https://sudact.ru/regular/doc/0jXKIhksC8Qx/', 'https://sudact.ru/regular/doc/d8GdLsdTZTgX/', 'https://sudact.ru/regular/doc/hDoZ73w99g1g/', 'https://sudact.ru/regular/doc/LqdfXu6vAB54/', 'https://sudact.ru/regular/doc/YTeSZhbaiL63/', 'https://sudact.ru/regular/doc/BhcC1qIPg7u/', 'https://sudact.ru/regular/doc/AQpQjhr3iAW8/', 'https://sudac

In [19]:
# res = cur.execute("SELECT id FROM decisions WHERE href = ?", (
#     current_url.replace('https://sudact.ru', ''),
# ))
# decision_id = res.fetchone()[0]

In [84]:
res = cur.execute("SELECT * FROM decisions WHERE href = ?;", (
    '/regular/doc/qWTZL4Q4Gkn/',
))
res.fetchone()

(3221,
 'Апелляционное постановление № 22К-1419/2020 УК-22-1419/2020 от 26 октября 2020 г. по делу № 3/2-203/2020',
 '/regular/doc/qWTZL4Q4Gkn/',
 52,
 'Калужский областной суд (Калужская область) - Уголовное\n\n\n',
 '\n\n...на представленных материалах дела, подтверждающих обоснованность ходатайства и принятого судом решения, не согласиться с данными выводами у суда апелляционной инстанции оснований не имеется. \nСогласно ст.  110   УПК РФ мера пресечения отменяется, когда в ней отпадает необходимость, или изменяется на более строгую или более мягкую, когда изменяются основания для избрания меры пресечения, предусмотренные статьями ...\n\n\n')

In [12]:
res = cur.execute("SELECT * FROM decisions LIMIT 1;")
res.fetchall()

[(1,
  'Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022',
  '/regular/doc/chaeDyxsKiYN/',
  32,
  'Шумихинский районный суд (Курганская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осужденного от отбывания назначенного наказания. Лицо, освобожденное от отбывания наказания по указанным основаниям, считается несудимым (часть 2 статьи 86 УК РФ).\nВ ...\n\n\n')]

In [441]:
# list(map(lambda x: x.replace('\n', '').replace('\t', '').replace('\xa0', ''), html.extract()))[3:-10]

['    Дело № 1- 13/2022',
 'Уникальный идентификатор дела: 45RS0023-01-2022-000129-83',
 'ПРИГОВОР',
 'Именем Российской Федерации',
 'город Шумиха 17 февраля 2022 года',
 'Шумихинский районный суд Курганской области в составе председательствующего судьи Пономарева Д.В.,',
 'с участием государственного обвинителя Антонова А.В.,',
 'подсудимой ФИО1,',
 'защитника подсудимой адвоката Недомец Е.В.,',
 'при секретаре Павловой А.В.,',
 'рассмотрев в открытом судебном заседании в г. Шумихе Шумихинского района Курганской области уголовное дело в отношении',
 'ФИО1, несудимой,',
 'обвиняемой в совершении преступления, предусмотренного п. «г» ч. 3 ст. ',
 '158 УК РФ',
 ',',
 'УСТАНОВИЛ:',
 'ФИО1 совершила кражу, то есть тайное хищение чужого имущества, совершенную с банковского счета, при следующих обстоятельствах.',
 'В период c 03 сентября по 05 сентября 2021 года ФИО1 в г. Шумихе Шумихинского района Курганской области, воспользовавшись тем, что в ее фактическом владении находится банковская 

In [445]:
# print('\n'.join(
#     list(
#         map(
#             lambda x: " ".join(
#                         x.replace('\n', '').
#                         replace('\t', '').
#                         replace('\xa0', '').
#                         split()
#                     ), 
#             html.extract()
#         )
#     )[3:-10]
# ))

Дело № 1- 13/2022
Уникальный идентификатор дела: 45RS0023-01-2022-000129-83
ПРИГОВОР
Именем Российской Федерации
город Шумиха 17 февраля 2022 года
Шумихинский районный суд Курганской области в составе председательствующего судьи Пономарева Д.В.,
с участием государственного обвинителя Антонова А.В.,
подсудимой ФИО1,
защитника подсудимой адвоката Недомец Е.В.,
при секретаре Павловой А.В.,
рассмотрев в открытом судебном заседании в г. Шумихе Шумихинского района Курганской области уголовное дело в отношении
ФИО1, несудимой,
обвиняемой в совершении преступления, предусмотренного п. «г» ч. 3 ст.
158 УК РФ
,
УСТАНОВИЛ:
ФИО1 совершила кражу, то есть тайное хищение чужого имущества, совершенную с банковского счета, при следующих обстоятельствах.
В период c 03 сентября по 05 сентября 2021 года ФИО1 в г. Шумихе Шумихинского района Курганской области, воспользовавшись тем, что в ее фактическом владении находится банковская карта АО «Почта Банк» на имя Л., ранее утерянная О. и самостоятельной матер

In [417]:
# class DecisionClearing(object):
#     def process_item(self, item, spider):
#         # удаляем рекламу
#         # удаляем скрипты
#         # html переносы заменяем на обычные (текстовые)
#         # удаляем остальные html теги        
        
#         print(item)
        
#         item['value'] = self.__remove_html_tags__(item['value'])
#         return item
#         # .css('::text').getall()
# #         lines = dict(item)["quote"].splitlines()
# #         first_line = self.__remove_html_tags__(lines[0])
# #         return {'quote': first_line}

#     def __remove_html_tags__(self, text):
#         """remove html tags from string"""
#         html_tags = re.compile('<.*?>')
#         return re.sub(html_tags, '', text)

In [184]:
# res = cur.execute("SELECT href FROM subsections;")
# res.fetchall()

In [185]:
# current_url = 'https://sudact.ru/practice/amnistiya/'

In [186]:
# current_url.split('?')[0]

In [187]:
# res = cur.execute("SELECT id FROM subsections WHERE href = ?", (current_url.replace('https://sudact.ru', '').split('?')[0],))
# subsection_id = res.fetchone()
# print(subsection_id)

In [188]:
# 'https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=5'.split('?')[0]

In [114]:
#cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cur.fetchall())

In [82]:
# res = cur.execute("SELECT id FROM sections WHERE href = ?", ('https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/',))
# section_id = res.fetchone()
# section_id

In [ ]:
            
# class ExtractFirstLine(object):
#     def process_item(self, item, spider):
#         """text processing"""
#         lines = dict(item)["quote"].splitlines()
#         #print(lines)
#         first_line = self.__remove_html_tags__(lines[0])

#         return {'quote': first_line}

#     def __remove_html_tags__(self, text):
#         """remove html tags from string"""
#         html_tags = re.compile('<.*?>')
#         return re.sub(html_tags, '', text)